## Imports

https://github.com/IBMDecisionOptimization/docplex-examples/blob/master/examples/mp/jupyter/tutorials/Linear_Programming.ipynb


Integer Optimization:
- https://github.com/IBMDecisionOptimization/docplex-examples/blob/master/examples/mp/jupyter/tutorials/Beyond_Linear_Programming.ipynb

In [121]:
from docplex.mp.model import Model
import numpy as np
import pandas as pd

In [122]:
# load data
players = pd.read_csv("../data/processed-data/0_player_selection.csv", index_col=0)
time = np.arange(10)
trials = np.arange(100)

# Set up all players
# use binary var cube
# (time, player, and simulation)


# set up goalkeeping separately
# set up DF, MF and Fw using separate dataset



In [124]:
# create model
ftcp = Model(name = "Rarita Football Team Composition")

max_age = 40
starting_budget = 15000000

In [125]:
# Decision variables
# assigned = ftcp.binary_var_matrix(keys1=players.index.values, keys2=time, keys3=trials, name = "assign_%s_%s_%s")
assigned = ftcp.binary_var_matrix(keys1=players.index.values, keys2=time, name = "assign_%s_%s")

# player_value = ftcp.binary_var_matrix(keys=players.index.values, keys2 = time, name = "mv_%s_%s")
# player_salary = ftcp.binary_var_matrix(keys=players.index.values, keys2 = time, name = "sal_%s_%s")

budget = ftcp.continuous_var_list(keys = time, name = "Budget_%s", lb=0)

In [126]:
# Non decision variables
# Total number of players
N_total = ftcp.integer_var_list(keys = time, name = 'Total number of Players_%s', lb=22, ub=25)

# Number of players in each role
N_gk = ftcp.integer_var_list(keys = time, name = "Number of goalkeepers_%s", lb=2, ub=3)
N_df = ftcp.integer_var_list(keys = time, name = "Number of defenders_%s", lb=8, ub=10)
N_mf = ftcp.integer_var_list(keys = time, name = "Number of midfielders_%s", lb=8, ub=10)
N_fw = ftcp.integer_var_list(keys = time, name = "Number of forwards_%s", lb=3, ub=4)       

In [127]:
# Set up Constraints for number of players
for year in time:
    
    previous_budget = starting_budget if year == 0 else budget[year-1]   
    ftcp.add_constraint(budget[year] == previous_budget - ftcp.sum(assigned[index, year] * player['Salary'] for index, player in players.iterrows()))
    
    ftcp.add_constraint(N_gk[year] == ftcp.sum(assigned[index, year] * player['GK'] for index, player in players.iterrows()))
    ftcp.add_constraint(N_df[year] == ftcp.sum(assigned[index, year] * player['DF'] for index, player in players.iterrows()))
    ftcp.add_constraint(N_mf[year] == ftcp.sum(assigned[index, year] * player['MF'] for index, player in players.iterrows()))
    ftcp.add_constraint(N_fw[year] == ftcp.sum(assigned[index, year] * player['FW'] for index, player in players.iterrows()))
    ftcp.add_constraint(N_gk[year] + N_df[year] + N_mf[year] + N_fw[year] == N_total[year])

    for index, player in players.iterrows():
        ftcp.add_constraint(assigned[index, year]*(player['Age'] + year) <= max_age )

In [128]:
ftcp.print_information()

Model: Rarita Football Team Composition
 - number of variables: 55600
   - binary=55540, integer=50, continuous=10
 - number of constraints: 55600
   - linear=55600
 - parameters: defaults
 - objective: none


In [129]:
ftcp.maximize(budget[9])
ftcp.solve(log_output=True)
ftcp.print_solution()

-- cannot find parameters matching version: , using: 20.1.0.0


DOcplexException: Cannot solve model: no CPLEX runtime found.